In [4]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [5]:
model = gp.Model("VRPLTT")

Restricted license - for non-production use only - expires 2023-10-25


In [6]:
Q_min = 140
Q_max = 300
n_levels = 4
load_levels = np.array([(0.5+i)*(Q_max-Q_min)/n_levels for i in range(n_levels)])
upper = load_levels+(Q_max-Q_min)/(n_levels*2)
lower = load_levels-(Q_max-Q_min)/(n_levels*2)

In [7]:
load_levels

array([ 20.,  60., 100., 140.])

In [8]:
upper

array([ 40.,  80., 120., 160.])

In [9]:

def vel(m, h, P):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eff = 0.95
    
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))/eff
    c3 = c_Fd/eff
    
    coefs = [c3, 0, c1, -P]
    #print(f"{np.real(np.roots(coefs))=}")
    v = np.max(np.real(3.6*np.roots(coefs)))
    return min(v,30)
    #return 3.6*np.roots(coefs)
    

def time_matrix(data_matrix, load_levels,P):
    N = len(data_matrix.index)
    elevation = data_matrix.elevation.to_numpy()
    d_ij_mat = data_matrix.iloc[:, 8:].to_numpy()
    t_ij_mat = np.zeros((N, N, n_levels))
    for i in range(N):
        for j in range(N):
            if i!=j:
                d_ij = d_ij_mat[i, j]
                h = (elevation[j] - elevation[i])/d_ij
                for l in range(n_levels):
                    m = load_levels[l]
                    t_ij_mat[i, j, l] =60*d_ij/vel(m, h, P)
    return np.round(t_ij_mat,4)

In [50]:
df = pd.read_csv("./instances/small/Fukuoka_01.csv")
P = 500 # Watt

t=time_matrix(df,load_levels,P=P)

In [51]:
from pprint import pprint
print(t[3,4])
print(t[4,3])


[1.1577 3.4176 5.6932 7.9698]
[0.332 0.332 0.332 0.332]


In [52]:
def extract_min(Q,distance):
    minimo=float("inf")
    nome=""
    for q in Q:
        if distance[q] <= minimo:
            nome=q
            minimo=distance[q]
    Q.remove(nome)
    return nome

def W(u,v,weight_matrix):
    for i in adj[u]:
        if i.end==v:
            return i.weight
        
def dijkstra(source,weight_matrix,l=0):
    distance=dict() #dict for distance from source
    parent=dict() #dict for parentnes of each node
    nodes=np.array([i for i in range(len(weight_matrix))]) #list of current node to iterate
    for node in nodes:
        distance[node]=float("inf") 
        parent[node]=None
    distance[source]=0
    S=[]
    Q={node for node in nodes} #set used ad que, coupled with distance dict
    while len(Q)>0:
        u=extract_min(Q,distance) #extract the min from Q,using distance dict
        S.append(u) #ultimated nodes
        for v,w in enumerate(weight_matrix[u]):
            if distance[v] > distance[u] + w[l]:
                distance[v]= distance[u] + w[l]
                parent[v]=u
        #print(f"{u = }")
        #print(f"{distance = }")
    return distance,parent
            
dist,parent = dijkstra(0,t)
pprint(dist)
pprint(parent)

{0: 0,
 1: 0.38,
 2: 1.4763000000000002,
 3: 2.5689,
 4: 3.3916,
 5: 3.6915999999999998,
 6: 4.0356,
 7: 5.315099999999999,
 8: 5.2126,
 9: 6.8701,
 10: 6.2101,
 11: 5.761,
 12: 3.2565999999999997,
 13: 3.1090999999999998,
 14: 4.5546,
 15: 6.0489999999999995,
 16: 6.0104999999999995,
 17: 2.3936,
 18: 1.9616,
 19: 1.1192,
 20: 2.1546000000000003}
{0: None,
 1: 0,
 2: 1,
 3: 2,
 4: 18,
 5: 4,
 6: 5,
 7: 13,
 8: 12,
 9: 10,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 12,
 15: 1,
 16: 14,
 17: 18,
 18: 0,
 19: 0,
 20: 19}


In [53]:
for i in t[:,:,0]:
    for j in i:
        print(f"\t{round(j,3)}",end=" ")
    print("\n")

	0.0 	0.38 	1.668 	3.389 	4.616 	5.555 	6.573 	8.238 	9.195 	9.47 	7.159 	6.709 	4.213 	3.624 	6.033 	6.711 	7.416 	2.495 	1.962 	1.119 	2.23 

	0.38 	0.0 	1.096 	2.315 	3.487 	4.474 	5.551 	7.112 	7.875 	8.143 	5.83 	5.381 	2.877 	2.729 	5.461 	5.669 	7.053 	3.121 	2.866 	1.551 	1.967 

	0.696 	0.36 	0.0 	1.093 	2.251 	3.265 	4.385 	5.896 	6.774 	7.549 	5.473 	4.738 	2.527 	2.372 	5.105 	5.312 	6.696 	2.794 	2.718 	0.824 	2.27 

	1.01 	0.674 	0.314 	0.0 	1.158 	2.157 	3.156 	4.8 	5.761 	6.535 	5.564 	3.723 	2.73 	3.162 	5.571 	5.669 	7.032 	1.488 	3.677 	1.138 	0.988 

	1.342 	1.006 	0.646 	0.332 	0.0 	0.3 	0.646 	3.621 	4.978 	6.549 	5.578 	3.737 	3.89 	4.318 	6.72 	6.83 	8.19 	1.82 	1.43 	1.47 	1.32 

	1.642 	1.306 	0.946 	0.632 	0.99 	0.0 	0.344 	2.62 	3.932 	5.654 	4.752 	4.318 	4.939 	5.361 	7.761 	6.508 	8.023 	2.122 	2.969 	1.772 	1.62 

	1.988 	1.65 	1.292 	0.978 	2.015 	0.938 	0.0 	1.433 	2.722 	4.598 	4.578 	4.723 	6.14 	6.553 	8.011 	6.333 	7.848 	2.466 	4.027 	2.116 	1.964

In [54]:
%%timeit
dist,parent = dijkstra(0,t)


256 µs ± 5.95 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
